# 使用api访问Embedding API

In [41]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv(".env.local"))

def get_embedding(text: str, model: str = None):
    # 获取API KEY
    client = OpenAI(
        api_key=os.getenv('ARK_API_KEY'),
        base_url=os.getenv('ARK_API_URL'),
    )

    if not model:
        model = os.getenv('ARK_EMBEDDING_MODEL')

    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response

response = get_embedding(text='要生成 embedding 的输入文本，字符串形式。')

In [42]:
response

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.70703125, 2.671875, 0.95703125, -3.484375, -0.421875, -4.03125, 1.4765625, 3.0625, 0.4609375, 0.80859375, -2.21875, 6.3125, 0.18359375, -5.03125, 4.03125, 2.296875, -5.15625, -2.421875, -4.75, -1.3828125, 3.921875, 1.7265625, -1.5703125, -0.9765625, 1.3359375, -0.9453125, -4.40625, -2.609375, -2.453125, 5.71875, 1.5078125, -0.54296875, -2.546875, 1.9609375, -2.765625, 3.734375, -1.90625, -11.5, -6.125, -0.447265625, -3.390625, 0.27734375, -1.828125, -1.2109375, 3.359375, 3.609375, -1.140625, -3.0, 6.125, -1.84375, -1.703125, 5.9375, -0.95703125, 0.220703125, 3.859375, 0.85546875, -0.29296875, 2.4375, 1.5546875, -5.15625, 4.625, -3.390625, 1.375, -1.53125, -3.421875, 3.8125, -2.828125, -3.125, -0.9609375, -0.384765625, -3.015625, -0.4765625, 2.078125, -2.140625, 0.1748046875, 0.609375, -2.296875, 2.109375, -5.0625, 3.390625, -1.2109375, 0.39453125, 0.419921875, 0.74609375, 0.95703125, -2.15625, -2.25, 1.53125, -0.1591796875, -3.90625

In [43]:
print(f'返回的embedding类型为：{response.object}')

返回的embedding类型为：list


In [44]:
print(f'embedding长度为：{len(response.data[0].embedding)}')
print(f'embedding（前10）为：{response.data[0].embedding[:10]}')

embedding长度为：4096
embedding（前10）为：[-0.70703125, 2.671875, 0.95703125, -3.484375, -0.421875, -4.03125, 1.4765625, 3.0625, 0.4609375, 0.80859375]


In [9]:
print(f'本次embedding model为：{response.model}')
print(f'本次token使用情况为：{response.usage}')

本次embedding model为：doubao-embedding-large-text-240915
本次token使用情况为：Usage(prompt_tokens=11, total_tokens=11)


# 1. 数据处理

## 1.1 数据加载

## 1.1.1 pdf数据处理

In [11]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf")

pdf_pages = loader.load()

In [12]:
print(f"载入后的变量类型为：{type(pdf_pages)}，",  f"该 PDF 一共包含 {len(pdf_pages)} 页")

载入后的变量类型为：<class 'list'>， 该 PDF 一共包含 196 页


In [13]:
pdf_page = pdf_pages[1]
print(f"每一个元素的类型：{type(pdf_page)}.", 
    f"该文档的描述性数据：{pdf_page.metadata}", 
    f"查看该文档的内容:\n{pdf_page.page_content}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'file_path': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'page': 1, 'total_pages': 196, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'xdvipdfmx (20200315)', 'creationDate': "D:20230303170709-00'00'", 'modDate': '', 'trapped': ''}
------
查看该文档的内容:
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我
等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二
下学生”。
使用说明
• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；
• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得
有点飘的时候再回来啃都来得及；

### 1.1.2 md数据处理

In [14]:
from langchain_community.document_loaders.markdown import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader("../data_base/knowledge_db/prompt_engineering/1. 简介 Introduction.md")
md_pages = loader.load()

In [15]:
print(f"载入后的变量类型为：{type(md_pages)}，",  f"该 Markdown 一共包含 {len(md_pages)} 页")

载入后的变量类型为：<class 'list'>， 该 Markdown 一共包含 1 页


In [16]:
md_page = md_pages[0]
print(f"每一个元素的类型：{type(md_page)}.", 
    f"该文档的描述性数据：{md_page.metadata}", 
    f"查看该文档的内容:\n{md_page.page_content[0:][:200]}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': '../data_base/knowledge_db/prompt_engineering/1. 简介 Introduction.md'}
------
查看该文档的内容:
第一章 简介

欢迎来到面向开发者的提示工程部分，本部分内容基于吴恩达老师的《Prompt Engineering for Developer》课程进行编写。《Prompt Engineering for Developer》课程是由吴恩达老师与 OpenAI 技术团队成员 Isa Fulford 老师合作授课，Isa 老师曾开发过受欢迎的 ChatGPT 检索插件，并且在教授 LLM （Larg


### 1.1.3 数据清洗

In [17]:
# 匹配"非中文字符+换行符+非中文字符"的模式，并将其中的换行符替换为空字符串
import re
pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
pdf_page.page_content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), pdf_page.page_content)
print(pdf_page.page_content)

前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我
等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二
下学生”。
使用说明
• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得
有点飘的时候再回来啃都来得及；• 每个公式的解析和推导我们都力(zhi) 争(neng) 以本科数学基础的视角进行讲解，所以超纲的数学知识
我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；• 若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们GitHub 的
Issues（地址：https://github.com/datawhalechina/pumpkin-book/issues）进行反馈，在对应版块
提交你希望补充的公式编号或者勘误信息，我们通常会在24 小时以内给您回复，超过24 小时未回复的
话可以微信联系我们（微信号：at-Sm1les）；
配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU
在线阅读地址：https://datawhalechina.github.io/pumpkin-book（仅供第1 版）
最新版PDF 获取地址：https://github.com/datawhalechina/pumpkin-book/releases
编委会
主

In [18]:
# 去除多余的字符
pdf_page.page_content = pdf_page.page_content.replace('•', '')
pdf_page.page_content = pdf_page.page_content.replace(' ', '')
print(pdf_page.page_content)

前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解,所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以......本南瓜书只能算是我
等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二
下学生”。
使用说明
南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；对于初学机器学习的小白，西瓜书第1章和第2章的公式强烈不建议深究，简单过一下即可，等你学得
有点飘的时候再回来啃都来得及；每个公式的解析和推导我们都力(zhi)争(neng)以本科数学基础的视角进行讲解，所以超纲的数学知识
我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们GitHub的
Issues（地址：https://github.com/datawhalechina/pumpkin-book/issues）进行反馈，在对应版块
提交你希望补充的公式编号或者勘误信息，我们通常会在24小时以内给您回复，超过24小时未回复的
话可以微信联系我们（微信号：at-Sm1les）；
配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU
在线阅读地址：https://datawhalechina.github.io/pumpkin-book（仅供第1版）
最新版PDF获取地址：https://github.com/datawhalechina/pumpkin-book/releases
编委会
主编：Sm1les、archwalker

In [19]:
# 去除连续的换行
md_page.page_content = md_page.page_content.replace('\n\n', '\n')
print(md_page.page_content)

第一章 简介
欢迎来到面向开发者的提示工程部分，本部分内容基于吴恩达老师的《Prompt Engineering for Developer》课程进行编写。《Prompt Engineering for Developer》课程是由吴恩达老师与 OpenAI 技术团队成员 Isa Fulford 老师合作授课，Isa 老师曾开发过受欢迎的 ChatGPT 检索插件，并且在教授 LLM （Large Language Model， 大语言模型）技术在产品中的应用方面做出了很大贡献。她还参与编写了教授人们使用 Prompt 的 OpenAI cookbook。我们希望通过本模块的学习，与大家分享使用提示词开发 LLM 应用的最佳实践和技巧。
网络上有许多关于提示词（Prompt， 本教程中将保留该术语）设计的材料，例如《30 prompts everyone has to know》之类的文章，这些文章主要集中在 ChatGPT 的 Web 界面上，许多人在使用它执行特定的、通常是一次性的任务。但我们认为，对于开发人员，大语言模型（LLM） 的更强大功能是能通过 API 接口调用，从而快速构建软件应用程序。实际上，我们了解到 DeepLearning.AI 的姊妹公司 AI Fund 的团队一直在与许多初创公司合作，将这些技术应用于诸多应用程序上。很兴奋能看到 LLM API 能够让开发人员非常快速地构建应用程序。
在本模块，我们将与读者分享提升大语言模型应用效果的各种技巧和最佳实践。书中内容涵盖广泛，包括软件开发提示词设计、文本总结、推理、转换、扩展以及构建聊天机器人等语言模型典型应用场景。我们衷心希望该课程能激发读者的想象力，开发出更出色的语言模型应用。
随着 LLM 的发展，其大致可以分为两种类型，后续称为基础 LLM 和指令微调（Instruction Tuned）LLM。基础LLM是基于文本训练数据，训练出预测下一个单词能力的模型。其通常通过在互联网和其他来源的大量数据上训练，来确定紧接着出现的最可能的词。例如，如果你以“从前，有一只独角兽”作为 Prompt ，基础 LLM 可能会继续预测“她与独角兽朋友共同生活在一片神奇森林中”。但是，如果你以“法国的首都是什么”为 Prompt ，则基础 LLM 可能会根据互联网上的文章，将回答预测为“法国最大的城市是什

## 1.2 文档分割

In [25]:
# 使用按字符串递归分割文本的分割器
# 分割优先级["\n\n", "\n", " ", ""])
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 单段文本长度
CHUNK_SIZE = 500

# 相邻文本重合长度
OVERLAP_SIZE = 50

# 初始化文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
# 分割文本
split_text = text_splitter.split_text(pdf_page.page_content[:1000])
print(len(split_text))
for t in split_text:
    print(len(t), "-"*100)
    print(t)

3
489 ----------------------------------------------------------------------------------------------------
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解,所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以......本南瓜书只能算是我
等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二
下学生”。
使用说明
南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；对于初学机器学习的小白，西瓜书第1章和第2章的公式强烈不建议深究，简单过一下即可，等你学得
489 ----------------------------------------------------------------------------------------------------
有点飘的时候再回来啃都来得及；每个公式的解析和推导我们都力(zhi)争(neng)以本科数学基础的视角进行讲解，所以超纲的数学知识
我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们GitHub的
Issues（地址：https://github.com/datawhalechina/pumpkin-book/issues）进行反馈，在对应版块
提交你希望补充的公式编号或者勘误信息，我们通常会在24小时以内给您回复，超过24小时未回复的
话可以微信联系我们（微信号：at

- 观察上述的文本会发现，并没有实现50个字符的重叠，这是由分割器决定的，该分割器会优先按自然结构进行划分（如换分、分段等）。

In [27]:
split_docs = text_splitter.split_documents(pdf_pages)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：711


In [29]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：305816


# 2. 搭建并使用向量数据库

## 2.1 加载知识库数据

In [31]:
# 递归获取所有知识库将要使用的文件路径
file_paths = []
folder_path = "../data_base/knowledge_db"
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.startswith("."):
            continue
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths[:3])

['../data_base/knowledge_db/prompt_engineering/6. 文本转换 Transforming.md', '../data_base/knowledge_db/prompt_engineering/4. 文本概括 Summarizing.md', '../data_base/knowledge_db/prompt_engineering/5. 推断 Inferring.md']


In [35]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader

# 构建数据加载器
loaders = []
for file_path in file_paths:
    file_type = file_path.split(".")[-1]
    if file_type == "pdf":
        loaders.append(PyMuPDFLoader(file_path))
    elif file_type == "md":
        loaders.append(UnstructuredMarkdownLoader(file_path))

# 加载数据
texts = []
for loader in loaders:
    texts.extend(loader.load())

In [36]:
# 查看部分数据
text = texts[1]
print(f"每一个元素的类型：{type(text)}.", 
    f"该文档的描述性数据：{text.metadata}", 
    f"查看该文档的内容:\n{text.page_content[0:]}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': '../data_base/knowledge_db/prompt_engineering/4. 文本概括 Summarizing.md'}
------
查看该文档的内容:
第四章 文本概括

在繁忙的信息时代，小明是一名热心的开发者，面临着海量的文本信息处理的挑战。他需要通过研究无数的文献资料来为他的项目找到关键的信息，但是时间却远远不够。在他焦头烂额之际，他发现了大型语言模型（LLM）的文本摘要功能。

这个功能对小明来说如同灯塔一样，照亮了他处理信息海洋的道路。LLM 的强大能力在于它可以将复杂的文本信息简化，提炼出关键的观点，这对于他来说无疑是巨大的帮助。他不再需要花费大量的时间去阅读所有的文档，只需要用 LLM 将它们概括，就可以快速获取到他所需要的信息。

通过编程调用 AP I接口，小明成功实现了这个文本摘要的功能。他感叹道：“这简直就像一道魔法，将无尽的信息海洋变成了清晰的信息源泉。”小明的经历，展现了LLM文本摘要功能的巨大优势：节省时间，提高效率，以及精准获取信息。这就是我们本章要介绍的内容，让我们一起来探索如何利用编程和调用API接口，掌握这个强大的工具。

一、单一文本概括

以商品评论的总结任务为例：对于电商平台来说，网站上往往存在着海量的商品评论，这些评论反映了所有客户的想法。如果我们拥有一个工具去概括这些海量、冗长的评论，便能够快速地浏览更多评论，洞悉客户的偏好，从而指导平台与商家提供更优质的服务。

接下来我们提供一段在线商品评价作为示例，可能来自于一个在线购物平台，例如亚马逊、淘宝、京东等。评价者为一款熊猫公仔进行了点评，评价内容包括商品的质量、大小、价格和物流速度等因素，以及他的女儿对该商品的喜爱程度。

python prod_review = """ 这个熊猫公仔是我给女儿的生日礼物，她很喜欢，去哪都带着。 公仔很软，超级可爱，面部表情也很和善。但是相比于价钱来说， 它有点小，我感觉在别的地方用同样的价钱能买到更大的。 快递比预期提前了一天到货，所以在送给女儿之前，我自己玩了会。 """

1.1 限制输出文本长度

我们首先尝试将文本的长度限制在30个

In [37]:
# 文档切分
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE,
)
split_docs = text_splitter.split_documents(texts)

## 2.2 构建chroma向量库

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv(".env.local"))

def get_embedding(text: str, model: str = None):
    # 获取API KEY
    client = OpenAI(
        api_key=os.getenv('ARK_API_KEY'),
        base_url=os.getenv('ARK_API_URL'),
    )

    if not model:
        model = os.getenv('ARK_EMBEDDING_MODEL')

    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response

response = get_embedding(text='要生成 embedding 的输入文本，字符串形式。')

In [47]:
# 封装Embedding以在langChain中使用
from typing import List
from langchain_core.embeddings import Embeddings
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

class ArkEmbeddings(Embeddings):

    def __init__(self, api_key: str, api_url: str, model: str):
        """
        实例化client.

        Args:
            api_key (str): 模型key 
            api_url (str): 模型url
            model (str): 模型名
        """


        self.client = OpenAI(
            api_key=api_key,
            base_url=api_url,
        )
        self.model = model
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embeddings.

        Args:
            texts (List[str]): 要生成 embedding 的文本列表.
        
        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表. 每个 embedding 都表示为一个浮点值列表.
        """
        result = []
        # 每批处理文本
        for i in range(0, len(texts), 64):
            embeddings = self.client.embeddings.create(
                model=self.model,
                input=texts[i:i+64]
            )
            result.extend([embeddings.embedding for embeddings in embeddings.data])
        
        return result

    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.

        Args:
            texts (str): 要生成 embedding 的文本.

        Return:
            embeddings (List[float]): 输入文本的 embedding, 一个浮点值列表.
        """
        return self.embed_documents([text])[0]


In [38]:
# 清空存储目录并新建
import shutil

persist_directory = "../data_base/vector_db/chroma"

if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)
os.makedirs(persist_directory)

In [49]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embedding = ArkEmbeddings(
    api_key=os.getenv('ARK_API_KEY'),
    api_url=os.getenv('ARK_API_URL'),
    model=os.getenv('ARK_EMBEDDING_MODEL'),
)

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory
)

In [50]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：1004


## 2.3 向量检索

In [51]:
# 基于余弦相似度进行检索
question = "什么是大语言模型"

sim_docs = vectordb.similarity_search(question, k=3)
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [53]:
# 查看检索结果
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i+1}个内容：\n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第1个内容：
综上，语言模型的幻觉问题事关应用的可靠性与安全性。开发者有必要认识到这一缺陷（注：截至2023年7月），并采取Prompt优化等措施予以缓解，以开发出更加可信赖的语言模型应用。这也将是未来语言模型进化的重要方向之一。

注意：

关于反斜杠使用的说明：在本教程中，我们使用反斜杠 \ 来使文本适应屏幕大小以提高阅读体验，而没有用换行符 \n 。GPT-3 并不受换行符（newline charact
--------------
检索到的第2个内容：
5.1 综合样例
--------------
检索到的第3个内容：
具体来说，首先编写初版 Prompt，然后通过多轮调整逐步改进，直到生成了满意的结果。对于更复杂的应用，可以在多个样本上进行迭代训练，评估 Prompt 的平均表现。在应用较为成熟后，才需要采用在多个样本集上评估 Prompt 性能的方式来进行细致优化。因为这需要较高的计算资源。

总之，Prompt 工程师的核心是掌握 Prompt 的迭代开发和优化技巧，而非一开始就要求100%完美。通过不断调
--------------


In [63]:
# 基于L2距离进行检索
# 只有在明确指定collection_metadata={"hnsw:space": "cosine"}时才会使用余弦距离
sim_docs_with_scores = vectordb.similarity_search_with_score(question, k=10)

# 查看检索结果
for i, (sim_doc, score) in enumerate(sim_docs_with_scores):
    print(f"检索到的第{i+1}个内容, 相似性打分={score}：\n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第1个内容, 相似性打分=5461.279296875：
综上，语言模型的幻觉问题事关应用的可靠性与安全性。开发者有必要认识到这一缺陷（注：截至2023年7月），并采取Prompt优化等措施予以缓解，以开发出更加可信赖的语言模型应用。这也将是未来语言模型进化的重要方向之一。

注意：

关于反斜杠使用的说明：在本教程中，我们使用反斜杠 \ 来使文本适应屏幕大小以提高阅读体验，而没有用换行符 \n 。GPT-3 并不受换行符（newline characters）的影响，但在您调用其他大模型时，需额外考虑换行符是否会影响模型性能。

四、英文原版 Prompt

1.1 使用分隔符清晰地表示输入的不同部分
--------------
检索到的第2个内容, 相似性打分=5687.2041015625：
5.1 综合样例
--------------
检索到的第3个内容, 相似性打分=5856.32470703125：
具体来说，首先编写初版 Prompt，然后通过多轮调整逐步改进，直到生成了满意的结果。对于更复杂的应用，可以在多个样本上进行迭代训练，评估 Prompt 的平均表现。在应用较为成熟后，才需要采用在多个样本集上评估 Prompt 性能的方式来进行细致优化。因为这需要较高的计算资源。

总之，Prompt 工程师的核心是掌握 Prompt 的迭代开发和优化技巧，而非一开始就要求100%完美。通过不断调整试错，最终找到可靠适用的 Prompt 形式才是设计 Prompt 的正确方法。

读者可以在 Jupyter Notebook 上，对本章给出的示例进行实践，修改 Prompt 并观察不同输出，以深入理解 Prompt 迭代优化的过程。这会对进一步开发复杂语言模型应用提供很好的实践准备。

三、英文版

产品说明书
--------------
检索到的第4个内容, 相似性打分=5953.07763671875：
is to cherck chatGPT for spelling abilitty" # spelling ]
--------------
检索到的第5个内容, 相似性打分=5953.07763671875：
is to cherck chatGPT for spelling abilitty" # spelling ]
-------

In [54]:
# 基于最大边际相关性检索
mmr_docs = vectordb.max_marginal_relevance_search(question, k=3)

In [55]:
# 查看检索结果
for i, sim_doc in enumerate(mmr_docs):
    print(f"检索到的第{i+1}个内容：\n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第1个内容：
综上，语言模型的幻觉问题事关应用的可靠性与安全性。开发者有必要认识到这一缺陷（注：截至2023年7月），并采取Prompt优化等措施予以缓解，以开发出更加可信赖的语言模型应用。这也将是未来语言模型进化的重要方向之一。

注意：

关于反斜杠使用的说明：在本教程中，我们使用反斜杠 \ 来使文本适应屏幕大小以提高阅读体验，而没有用换行符 \n 。GPT-3 并不受换行符（newline charact
--------------
检索到的第2个内容：
5.1 综合样例
--------------
检索到的第3个内容：
is to cherck chatGPT for spelling abilitty" # spelling ]
--------------
